# Preparation

In [46]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets
from torch.utils.data.dataset import Subset
import matplotlib.pyplot as plt
import random
import os
import enum
import numpy as np
import pandas as pd
import collections

# Create dataset

In [47]:
class QuestionType(enum.Enum):
    WHICH_ONE = 'which_one'
    IS_IN = 'is_in'
class DataSet(enum.Enum):
    MNIST = 'mnist'
    FASHION = 'fashion'
    KMNIST = 'kmnist'
    CIFAR10 = 'cifar-10'

get_question_type = {QuestionType.WHICH_ONE: 'which_one',
                       QuestionType.IS_IN: 'is_in'}
get_dataset_name = {DataSet.MNIST: 'mnist',
                    DataSet.FASHION: 'fashion', 
                    DataSet.KMNIST: 'kmnist',
                    DataSet.CIFAR10: 'cifar-10'}

In [98]:
def prepare_mnist_data(batch_size):
    ordinary_train_dataset = datasets.MNIST(root='./data/mnist', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.MNIST(root='./data/mnist', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

def prepare_kmnist_data(batch_size):
    ordinary_train_dataset = datasets.KMNIST(root='./data/KMNIST', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.KMNIST(root='./data/KMNIST', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

def prepare_fashion_data(batch_size):
    ordinary_train_dataset = datasets.FashionMNIST(root='./data/FashionMnist', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.FashionMNIST(root='./data/FashionMnist', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=False, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

#torchvision.datasets.CIFAR10.url="http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
def prepare_cifar10_data(batch_size: int) -> [DataLoader, DataLoader, DataLoader, DataLoader]:
    train_transform = T.Compose(
        [T.ToTensor(), # transforms.RandomHorizontalFlip(), transforms.RandomCrop(32,4),
         T.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    test_transform = T.Compose(
        [T.ToTensor(),
         T.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=test_transform)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=len(train_dataset.data), shuffle=False, num_workers=0)
    return full_train_loader, train_loader, test_loader

full_train_loader_fashion, train_loader_fashion, test_loader_fashion = prepare_fashion_data(256)
datas_fashion, labels_fashion = next(iter(full_train_loader_fashion))
full_train_loader_kmnist, train_loader_kmnist, test_loader_kmnist = prepare_kmnist_data(256)
datas_kmnist, labels_kmnist = next(iter(full_train_loader_kmnist))
full_train_loader_mnist, train_loader_mnist, test_loader_mnist = prepare_mnist_data(256)
datas_mnist, labels_mnist = next(iter(full_train_loader_mnist))
full_train_loader_cifar, train_loader_cifar, test_loader_cifar = prepare_cifar10_data(256)
datas_cifar, labels_cifar = next(iter(full_train_loader_cifar))

Files already downloaded and verified


In [112]:
next(iter(full_train_loader_cifar))[0]

tensor([[[[-1.0527e+00, -1.3068e+00, -1.1956e+00,  ...,  5.1906e-01,
            4.2380e-01,  3.6029e-01],
          [-1.7354e+00, -1.9895e+00, -1.7037e+00,  ..., -3.6628e-02,
           -1.0013e-01, -5.2505e-02],
          [-1.5926e+00, -1.7354e+00, -1.2115e+00,  ..., -1.1601e-01,
           -8.4258e-02, -2.5890e-01],
          ...,
          [ 1.3129e+00,  1.2018e+00,  1.1541e+00,  ...,  5.5081e-01,
           -1.1004e+00, -1.1480e+00],
          [ 8.6835e-01,  7.5721e-01,  9.6361e-01,  ...,  9.3186e-01,
           -4.4942e-01, -6.7170e-01],
          [ 8.2072e-01,  6.7783e-01,  8.5247e-01,  ...,  1.4399e+00,
            4.0792e-01, -3.6628e-02]],

         [[-9.8380e-01, -1.2420e+00, -1.2097e+00,  ...,  1.4587e-01,
            3.2906e-02,  1.6768e-02],
          [-1.6616e+00, -1.9844e+00, -1.8553e+00,  ..., -5.6421e-01,
           -6.4490e-01, -5.8034e-01],
          [-1.5970e+00, -1.8714e+00, -1.5486e+00,  ..., -6.2876e-01,
           -6.2876e-01, -8.0628e-01],
          ...,
     

In [178]:
def get_random_dataloader(full_train_loader: DataLoader, batch_size: int) -> [DataLoader, DataLoader]:
    datas, labels = next(iter(full_train_loader))
    count_list, delete_index_list = [0 for i in range(10)], []
    index_list = list(range(len(datas)))
    #random.shuffle(index_list)
    for index in index_list:
        target = labels[index].tolist()
        if count_list[target] >= 1000:
            delete_index_list.append(index)
        else:
            count_list[target] += 1
    datas = np.delete(datas, delete_index_list, 0)
    labels = np.delete(labels, delete_index_list, 0)
    
    random_train_matrix_dataset = torch.utils.data.TensorDataset(datas, labels.float())
    full_random_train_loader = torch.utils.data.DataLoader(dataset=random_train_matrix_dataset, batch_size=labels.shape[0], shuffle=False, num_workers=0)
    random_train_loader = torch.utils.data.DataLoader(dataset=random_train_matrix_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return full_random_train_loader, random_train_loader
full_random_train_loader_cifar, random_train_loader_cifar = get_random_dataloader(full_train_loader_cifar, 64)

In [179]:
def generate_Q_A_label(single_class_assigned: int, 
                         question_type: int, 
                         all_class_size: int, 
                         question_class_size: int) -> list:
    
    all_class_set = range(all_class_size)
    question_class_set = random.sample(all_class_set, question_class_size)
    temporary_label_list = [0 for i in range(all_class_size)]
    
    if question_type == QuestionType.WHICH_ONE:
        if single_class_assigned in question_class_set:
            for i in [single_class_assigned]:
                temporary_label_list[i] = 1
        
        else:
            comp_question_class_set = [i for i in all_class_set if i not in question_class_set]
            for i in comp_question_class_set:
                temporary_label_list[i] = 1
        
    elif question_type == QuestionType.IS_IN:
        if single_class_assigned in question_class_set:
            for i in question_class_set:
                temporary_label_list[i] = 1
        
        else:
            comp_question_class_set = [i for i in all_class_set if i not in question_class_set]
            for i in comp_question_class_set:
                temporary_label_list[i] = 1
    
    return temporary_label_list

temporary_label_list = generate_Q_A_label(single_class_assigned=0, 
                                            question_type=QuestionType.WHICH_ONE, 
                                            all_class_size=10, 
                                            question_class_size=9)

temporary_label_list

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [180]:
def generate_Q_A_labels(labels_tensor: torch.tensor, 
                          question_type: QuestionType,
                          all_class_size: int,
                          question_class_size: int) -> torch.tensor:
    
    labels_list = [int(i) for i in labels_tensor.tolist()]
    Q_A_labels_list = []
    
    for label in labels_list:
        Q_A_label = generate_Q_A_label(single_class_assigned=label, 
                                          question_type=question_type,  # change out of this function
                                          all_class_size=all_class_size, # change out of this function
                                          question_class_size=question_class_size) # change out of this function
        Q_A_labels_list.append(Q_A_label)
        
    Q_A_labels_tensor = torch.tensor(Q_A_labels_list)
    
    return Q_A_labels_tensor

In [181]:
def generate_dataloader_with_Q_A_label(full_random_train_loader: DataLoader, 
                                       question_type: QuestionType,
                                       question_size: int,
                                       batch_size: int) ->[DataLoader, int, int]:
    
    for i, (datas, labels) in enumerate(full_random_train_loader):
        all_class_size = torch.max(labels) + 1 # K is number of classes, full_train_loader is full batch
        all_class_size = int(all_class_size.tolist())
        
    Q_A_labels_tensor = generate_Q_A_labels(labels, question_type, all_class_size, question_size)
    Q_A_labels_matrix_dataset = torch.utils.data.TensorDataset(datas, Q_A_labels_tensor.float())
    
    Q_A_labels_matrix_train_loader = torch.utils.data.DataLoader(dataset=Q_A_labels_matrix_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_Q_A_labels_matrix_train_loader = torch.utils.data.DataLoader(dataset=Q_A_labels_matrix_dataset, batch_size=datas.shape[0], shuffle=False, num_workers=0)
    dimension = int(datas.reshape(-1).shape[0]/datas.shape[0])
    return full_Q_A_labels_matrix_train_loader, Q_A_labels_matrix_train_loader, dimension, all_class_size

In [185]:
from traitlets.traitlets import Integer
# Unclear!!!
DataSet_2_DataLoader_generator = {
    DataSet.MNIST: prepare_mnist_data,
    DataSet.FASHION: prepare_fashion_data,
    DataSet.KMNIST: prepare_kmnist_data,
    DataSet.CIFAR10: prepare_cifar10_data
}

def preparence(dataset: DataSet, batch_size: int):
    dataloader_generator = DataSet_2_DataLoader_generator[dataset]
    full_train_loader, train_loader, test_loader = dataloader_generator(batch_size)
    full_random_train_loader, random_train_loader = get_random_dataloader(full_train_loader, batch_size)
    
    return full_random_train_loader, random_train_loader, test_loader

full_random_train_loader, random_train_loader, test_loader = preparence(DataSet.CIFAR10, 256)

Files already downloaded and verified


In [186]:
next(iter(full_random_train_loader))

[tensor([[[[-1.0527e+00, -1.3068e+00, -1.1956e+00,  ...,  5.1906e-01,
             4.2380e-01,  3.6029e-01],
           [-1.7354e+00, -1.9895e+00, -1.7037e+00,  ..., -3.6628e-02,
            -1.0013e-01, -5.2505e-02],
           [-1.5926e+00, -1.7354e+00, -1.2115e+00,  ..., -1.1601e-01,
            -8.4258e-02, -2.5890e-01],
           ...,
           [ 1.3129e+00,  1.2018e+00,  1.1541e+00,  ...,  5.5081e-01,
            -1.1004e+00, -1.1480e+00],
           [ 8.6835e-01,  7.5721e-01,  9.6361e-01,  ...,  9.3186e-01,
            -4.4942e-01, -6.7170e-01],
           [ 8.2072e-01,  6.7783e-01,  8.5247e-01,  ...,  1.4399e+00,
             4.0792e-01, -3.6628e-02]],
 
          [[-9.8380e-01, -1.2420e+00, -1.2097e+00,  ...,  1.4587e-01,
             3.2906e-02,  1.6768e-02],
           [-1.6616e+00, -1.9844e+00, -1.8553e+00,  ..., -5.6421e-01,
            -6.4490e-01, -5.8034e-01],
           [-1.5970e+00, -1.8714e+00, -1.5486e+00,  ..., -6.2876e-01,
            -6.2876e-01, -8.0628e-01],


In [187]:
def transform_scalar_to_array(labels: torch.tensor, all_class_size: int) -> torch.Tensor:
    outputs = []
    for label in labels:
        label = int(label.tolist())
        label_full_size = [0] * all_class_size
        label_full_size[label] = 1
        outputs.append(label_full_size)
    
    outputs = torch.tensor(outputs)
    
    return outputs

# Model excution

In [188]:
import torch.nn as nn
import torch
import math
import torch.nn.functional as F

def mae_loss(outputs, labels):
    sm_outputs = F.softmax(outputs, dim=1)
    loss_fn = nn.L1Loss(reduction='none')
    loss_matrix = loss_fn(sm_outputs, labels.float())
    sample_loss = loss_matrix.sum(dim=-1)
    return sample_loss
    
def mse_loss(outputs, labels):
    sm_outputs = F.softmax(outputs, dim=1)
    loss_fn = nn.MSELoss(reduction='none')
    loss_matrix = loss_fn(sm_outputs, labels.float())
    sample_loss = loss_matrix.sum(dim=-1)
    return sample_loss

def gce_loss(outputs, labels):
    q = 0.7
    sm_outputs = F.softmax(outputs, dim=1)
    pow_outputs = torch.pow(sm_outputs, q)
    sample_loss = (1-(pow_outputs*labels).sum(dim=1))/q # n
    return sample_loss

def phuber_ce_loss(outputs, labels):
    trunc_point = 0.1
    n = labels.shape[0]
    soft_max = nn.Softmax(dim=1)
    sm_outputs = soft_max(outputs)
    final_outputs = sm_outputs * labels
    final_confidence = final_outputs.sum(dim=1)
    ce_index = (final_confidence > trunc_point)
    sample_loss = torch.zeros(n).to(device)

    if ce_index.sum() > 0:
        ce_outputs = outputs[ce_index,:]
        logsm = nn.LogSoftmax(dim=-1)
        logsm_outputs = logsm(ce_outputs)
        final_ce_outputs = logsm_outputs * labels[ce_index,:]
        sample_loss[ce_index] = - final_ce_outputs.sum(dim=-1)

    linear_index = (final_confidence <= trunc_point)

    if linear_index.sum() > 0:
        sample_loss[linear_index] = -math.log(trunc_point) + (-1/trunc_point)*final_confidence[linear_index] + 1

    return sample_loss

def ce_loss(outputs, labels):
    logsm = nn.LogSoftmax(dim=1)
    logsm_outputs = logsm(outputs)
    final_outputs = logsm_outputs * labels
    sample_loss = - final_outputs.sum(dim=1)
    return sample_loss

def W_O_loss(loss_fn_, outputs, labels, device, question_class_size, all_class_size):
    n, k = labels.shape[0], labels.shape[1]
    temp_loss = torch.zeros(n, k).to(device)
    for i in range(k):
        tempY = torch.zeros(n, k).to(device)
        tempY[:, i] = 1.0
        temp_loss[:, i] = loss_fn_(outputs, tempY)
        
    candidate_loss = (temp_loss * labels).sum(dim=1)
    noncandidate_loss = (temp_loss * (1-labels)).sum(dim=1)
    total_loss = candidate_loss - ((all_class_size - question_class_size) * (all_class_size - question_class_size - 1))/(question_class_size * (2*all_class_size - question_class_size - 1.0)) * noncandidate_loss
    average_loss = total_loss.mean()
    return average_loss

def I_I_loss(loss_fn_, outputs, labels, device, question_class_size, all_class_size):
    n, k = labels.shape[0], labels.shape[1]
    temp_loss = torch.zeros(n, k).to(device)
    for i in range(k):
        tempY = torch.zeros(n, k).to(device)
        tempY[:, i] = 1.0
        temp_loss[:, i] = loss_fn_(outputs, tempY)
        
    candidate_loss = (temp_loss * labels).sum(dim=1)
    noncandidate_loss = (temp_loss * (1-labels)).sum(dim=1)
    total_loss = candidate_loss - (2*question_class_size**2 + all_class_size**2 - all_class_size*(2*question_class_size + 1))/(2*question_class_size * (all_class_size - question_class_size)) * noncandidate_loss
    average_loss = total_loss.mean()
    return average_loss


In [189]:
class mlp_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(mlp_model, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = x.view(-1, self.num_flat_features(x))
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [190]:
# ResNet-18 を作成する。
resnet = models.resnet18(pretrained=True)

# 出力層の出力数を ImageNet の 1000 からこのデータセットのクラス数である 2 に置き換える。
resnet.fc = nn.Linear(resnet.fc.in_features, 10)


In [191]:
def accuracy_check(loader, model, device):
    with torch.no_grad():
        total, num_samples = 0, 0
        for images, labels in loader:
            labels, images = labels.to(device), images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += (predicted == labels).sum().item()
            num_samples += labels.size(0) 
    return total / num_samples

In [192]:
def loss_check(loader, model, criterion, device, all_class_size):
    with torch.no_grad():
        total, num_samples = 0, 0
        for images, labels in loader:
            labels, images = labels.to(device), images.to(device)
            outputs = model(images)
            labels = transform_scalar_to_array(labels, all_class_size)
            labels = labels.to(device)
            loss = criterion(outputs,labels)
            total += loss.sum().item()
            num_samples += labels.size(0) 
    return total / num_samples

In [193]:
def show_loss(epoch,max_epoch, loss):
    print('TRAIN EPOCH[{:03}/{:03}] LOSS:{:03f}%'.format(epoch, max_epoch, loss))    
def show_eval_loss(epoch,max_epoch, loss, is_val):
    if is_val:
        print('EVAL TEST EPOCH[{:03}/{:03}] LOSS:{:03f}%'.format(epoch, max_epoch, loss))
    else:
        print('EVAL TRAIN EPOCH[{:03}/{:03}] LOSS:{:03f}%'.format(epoch, max_epoch, loss))
def show_acc(epoch,max_epoch, acc, is_val):
    if is_val:
        print('TEST EPOCH[{:03}/{:03}] ACC:{:03f}%'.format(epoch, max_epoch, acc*100))
    else:
        print('TRAIN EPOCH[{:03}/{:03}] ACC:{:03f}%'.format(epoch, max_epoch, acc*100))

In [194]:
def Train(model, 
          question_type, 
          question_class_size, 
          all_class_size, 
          criterion, 
          optimizer, 
          data_loader, 
          device, 
          epoch, 
          max_epoch):
        
    total_loss_train = 0.0
    model.train()

    for n, (data, label) in enumerate(data_loader):
        optimizer.zero_grad()
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        if question_type == QuestionType.WHICH_ONE:
            loss = W_O_loss(criterion, output, label.float(), device, question_class_size, all_class_size)
        elif question_type == QuestionType.IS_IN:
            loss = I_I_loss(criterion, output, label.float(), device, question_class_size, all_class_size)
        else:
            loss = criterion(output,label)

        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()

    show_loss(epoch+1, max_epoch, total_loss_train/(n+1))
    print()

    return total_loss_train/(n+1), model

In [195]:
def Eval(index,
        model, 
        evaluation_data_loader_train, 
        evaluation_data_loader_test,
        all_class_size,
        criterion,
        device, 
        epoch, 
        max_epoch):
    
    model.eval()
    if index == 'acc':
        total_acc_train = accuracy_check(evaluation_data_loader_train, model, device)
        total_acc_test = accuracy_check(evaluation_data_loader_test, model, device)

        show_acc(epoch+1, max_epoch, total_acc_train, is_val=False)
        print()
        show_acc(epoch+1, max_epoch, total_acc_test, is_val=True)
        print()

        return total_acc_train , total_acc_test, model
    elif index == 'loss':
        total_loss_train = loss_check(evaluation_data_loader_train, model, criterion, device, all_class_size)
        total_loss_test = loss_check(evaluation_data_loader_test, model, criterion, device, all_class_size)
        show_eval_loss(epoch+1, max_epoch, total_loss_train, is_val=False)
        print()
        show_eval_loss(epoch+1, max_epoch, total_loss_test, is_val=True)
        print()

        return total_loss_train , total_loss_test, model
        

In [196]:
get_loss_function = {ce_loss: 'CE_LOSS', 
                     mae_loss: 'MAE_LOSS'}

In [197]:
from tqdm.notebook import tqdm
import collections

In [231]:
def Q_A_label_Train_Eval(index,
                          dataset, 
                          question_type, 
                          question_class_size, 
                          model_name, 
                          batch_size, 
                          loss_fn, 
                          EPOCHS):
    print(f'START: {question_type}, {question_class_size}')
    
    full_Q_A_labels_matrix_train_loader, Q_A_labels_matrix_train_loader, dimension, all_class_size = generate_dataloader_with_Q_A_label(full_random_train_loader,
                                                                                                                                       question_type,
                                                                                                                                       question_class_size,
                                                                                                                                       batch_size)
    full_Q_A_datas, full_Q_A_labels = next(iter(full_Q_A_labels_matrix_train_loader))
    counter = collections.Counter(torch.sum(full_Q_A_labels, 1).int().tolist())
    print(counter)
    
    
    DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    #DEVICE = torch.device('mps')
    index=index
    
    if model_name == 'MLP':
        model = mlp_model(input_dim=dimension, hidden_dim=500, output_dim=all_class_size).to(DEVICE)
    elif model_name == 'ResNet':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 10)
        
    model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

    train_index_list = []
    eval_train_index_list = []
    eval_test_index_list = []
    #results_df = pd.DataFrame(columns=["index", "dataset", "epoch", "question_type", "question_class_size", "model", "loss_function", "train_index", "train_index", "test_index"])
    results_df = []
    for epoch in tqdm(range(EPOCHS)):
        train_index, model = Train(model=model, 
                                  question_type=question_type,
                                  question_class_size=question_class_size,
                                  all_class_size=all_class_size,
                                  criterion=loss_fn,
                                  optimizer=optimizer, 
                                  data_loader=Q_A_labels_matrix_train_loader,
                                  device=DEVICE, 
                                  epoch=epoch, 
                                  max_epoch=EPOCHS)
        
        eval_train_index, eval_test_index, model = Eval(index=index,
                                                        model=model, 
                                                        evaluation_data_loader_train=random_train_loader, 
                                                        evaluation_data_loader_test=test_loader,
                                                        all_class_size=all_class_size,
                                                        criterion=loss_fn,
                                                        device=DEVICE, 
                                                        epoch=epoch, 
                                                        max_epoch=EPOCHS)

        train_index_list.append(train_index)
        eval_train_index_list.append(eval_train_index)
        eval_test_index_list.append(eval_test_index)

        result_df = pd.DataFrame({'index': [index],
                                      'dataset': [get_dataset_name[dataset]],
                                      'epoch': [epoch+1],
                                      'question_type': [get_question_type[question_type]], 
                                      'question_class_size': [question_class_size], 
                                      'model': [model_name],
                                      'loss_function': [get_loss_function[loss_fn]],
                                      'train_index': [train_index],
                                      'eval_train_index': [eval_train_index],
                                      'eval_test_index': [eval_test_index]})
        results_df.append(result_df)
        
        if epoch == 150:
            results_df_tmp = pd.concat(results_df, axis=0)
            results_df_tmp.to_csv(f'result_{get_dataset_name[dataset]}_{question_type}_{question_class_size}.csv', index=None)
    
    results_df = pd.concat(results_df, axis=0)
    
    if question_type == QuestionType.WHICH_ONE:
        if question_class_size != 9: 
            results_df['label_size_cand'] = counter[1]
            results_df['label_size_comp'] = counter[all_class_size - question_class_size]
        else:
            results_df['label_size_cand'] = counter[1]
            results_df['label_size_comp'] = np.nan
    elif question_type == QuestionType.IS_IN:
        if question_class_size != 5: 
            results_df['label_size_cand'] = counter[question_class_size]
            results_df['label_size_comp'] = counter[all_class_size - question_class_size]
        else:
            results_df['label_size_cand'] = counter[question_class_size]
            results_df['label_size_comp'] = np.nan
    return results_df

In [232]:
index = 'acc' #'acc' or 'loss'
for dataset_name in tqdm([DataSet.CIFAR10]):#DataSet.MNIST or DataSet.KMNIST or DataSet.FASHION or Dataset.CIFAR10
    results = []
    full_random_train_loader, random_train_loader, test_loader = preparence(dataset_name, 512)
    model_name = 'ResNet' #'MLP' or 'ResNet'
    for itr in tqdm(range(1,6)):
        for question_type in tqdm([QuestionType.IS_IN]):
            for question_size in tqdm(range(1,6)):
                    result_df = Q_A_label_Train_Eval(index=index,
                                                     dataset=dataset_name, 
                                                     question_type=question_type, 
                                                     question_class_size=question_size, 
                                                     model_name=model_name, 
                                                     batch_size=800, 
                                                     loss_fn=mae_loss, 
                                                     EPOCHS=150)
                    results.append(result_df)
    output = pd.concat(results, axis=0)
    output.to_csv(f'all_result_{get_dataset_name[dataset_name]}.csv', index=None)

Files already downloaded and verified


START: QuestionType.IS_IN, 1
Counter({9: 8997, 1: 1003})


TRAIN EPOCH[001/150] LOSS:1.472426%

TRAIN EPOCH[001/150] ACC:25.770000%

TEST EPOCH[001/150] ACC:25.210000%

TRAIN EPOCH[002/150] LOSS:0.919932%

TRAIN EPOCH[002/150] ACC:42.670000%

TEST EPOCH[002/150] ACC:41.040000%

TRAIN EPOCH[003/150] LOSS:0.736582%

TRAIN EPOCH[003/150] ACC:46.130000%

TEST EPOCH[003/150] ACC:43.540000%

TRAIN EPOCH[004/150] LOSS:0.664891%

TRAIN EPOCH[004/150] ACC:46.320000%

TEST EPOCH[004/150] ACC:44.740000%

TRAIN EPOCH[005/150] LOSS:0.638211%

TRAIN EPOCH[005/150] ACC:48.750000%

TEST EPOCH[005/150] ACC:46.340000%

TRAIN EPOCH[006/150] LOSS:0.629789%

TRAIN EPOCH[006/150] ACC:49.430000%

TEST EPOCH[006/150] ACC:46.440000%

TRAIN EPOCH[007/150] LOSS:0.621406%

TRAIN EPOCH[007/150] ACC:49.550000%

TEST EPOCH[007/150] ACC:46.780000%

TRAIN EPOCH[008/150] LOSS:0.653803%

TRAIN EPOCH[008/150] ACC:50.010000%

TEST EPOCH[008/150] ACC:47.420000%

TRAIN EPOCH[009/150] LOSS:0.614477%

TRAIN EPOCH[009/150] ACC:46.800000%

TEST EPOCH[009/150] ACC:45.300000%

TRAIN EPOC

TRAIN EPOCH[076/150] LOSS:0.209042%

TRAIN EPOCH[076/150] ACC:61.130000%

TEST EPOCH[076/150] ACC:56.180000%

TRAIN EPOCH[077/150] LOSS:0.207919%

TRAIN EPOCH[077/150] ACC:60.540000%

TEST EPOCH[077/150] ACC:55.460000%

TRAIN EPOCH[078/150] LOSS:0.216479%

TRAIN EPOCH[078/150] ACC:59.830000%

TEST EPOCH[078/150] ACC:54.820000%

TRAIN EPOCH[079/150] LOSS:0.225476%

TRAIN EPOCH[079/150] ACC:59.760000%

TEST EPOCH[079/150] ACC:54.760000%

TRAIN EPOCH[080/150] LOSS:0.207651%

TRAIN EPOCH[080/150] ACC:57.840000%

TEST EPOCH[080/150] ACC:53.460000%

TRAIN EPOCH[081/150] LOSS:0.196916%

TRAIN EPOCH[081/150] ACC:59.940000%

TEST EPOCH[081/150] ACC:55.340000%

TRAIN EPOCH[082/150] LOSS:0.179916%

TRAIN EPOCH[082/150] ACC:60.520000%

TEST EPOCH[082/150] ACC:55.960000%

TRAIN EPOCH[083/150] LOSS:0.173398%

TRAIN EPOCH[083/150] ACC:60.880000%

TEST EPOCH[083/150] ACC:56.990000%

TRAIN EPOCH[084/150] LOSS:0.186079%

TRAIN EPOCH[084/150] ACC:60.640000%

TEST EPOCH[084/150] ACC:56.620000%

TRAIN EPOC

Counter({8: 7979, 2: 2021})


...

TRAIN EPOCH[001/150] LOSS:1.261108%

TRAIN EPOCH[001/150] ACC:41.830000%

TEST EPOCH[001/150] ACC:41.550000%

TRAIN EPOCH[002/150] LOSS:0.704976%

TRAIN EPOCH[002/150] ACC:53.350000%

TEST EPOCH[002/150] ACC:51.170000%

TRAIN EPOCH[003/150] LOSS:0.533769%

TRAIN EPOCH[003/150] ACC:62.990000%

TEST EPOCH[003/150] ACC:58.100000%

TRAIN EPOCH[004/150] LOSS:0.483087%

TRAIN EPOCH[004/150] ACC:63.630000%

TEST EPOCH[004/150] ACC:59.590000%

TRAIN EPOCH[005/150] LOSS:0.455100%

TRAIN EPOCH[005/150] ACC:62.230000%

TEST EPOCH[005/150] ACC:56.630000%

TRAIN EPOCH[006/150] LOSS:0.461692%

TRAIN EPOCH[006/150] ACC:60.280000%

TEST EPOCH[006/150] ACC:55.570000%

TRAIN EPOCH[007/150] LOSS:0.412866%

TRAIN EPOCH[007/150] ACC:62.990000%

TEST EPOCH[007/150] ACC:57.790000%

TRAIN EPOCH[008/150] LOSS:0.396364%

TRAIN EPOCH[008/150] ACC:65.630000%

TEST EPOCH[008/150] ACC:59.460000%

TRAIN EPOCH[009/150] LOSS:0.393833%

TRAIN EPOCH[009/150] ACC:69.460000%

TEST EPOCH[009/150] ACC:62.300000%

TRAIN EPOC

TRAIN EPOCH[076/150] LOSS:0.020510%

TRAIN EPOCH[076/150] ACC:77.690000%

TEST EPOCH[076/150] ACC:67.290000%

TRAIN EPOCH[077/150] LOSS:0.017011%

TRAIN EPOCH[077/150] ACC:77.630000%

TEST EPOCH[077/150] ACC:66.070000%

TRAIN EPOCH[078/150] LOSS:0.015714%

TRAIN EPOCH[078/150] ACC:77.580000%

TEST EPOCH[078/150] ACC:66.720000%

TRAIN EPOCH[079/150] LOSS:0.013160%

TRAIN EPOCH[079/150] ACC:77.940000%

TEST EPOCH[079/150] ACC:67.040000%

TRAIN EPOCH[080/150] LOSS:0.012800%

TRAIN EPOCH[080/150] ACC:77.910000%

TEST EPOCH[080/150] ACC:66.990000%

TRAIN EPOCH[081/150] LOSS:0.015015%

TRAIN EPOCH[081/150] ACC:77.940000%

TEST EPOCH[081/150] ACC:66.850000%

TRAIN EPOCH[082/150] LOSS:0.008253%

TRAIN EPOCH[082/150] ACC:79.060000%

TEST EPOCH[082/150] ACC:67.980000%

TRAIN EPOCH[083/150] LOSS:0.002748%

TRAIN EPOCH[083/150] ACC:79.020000%

TEST EPOCH[083/150] ACC:68.190000%

TRAIN EPOCH[084/150] LOSS:-0.001005%

TRAIN EPOCH[084/150] ACC:79.090000%

TEST EPOCH[084/150] ACC:68.460000%

TRAIN EPO

TRAIN EPOCH[150/150] ACC:78.360000%

TEST EPOCH[150/150] ACC:66.960000%


START: QuestionType.IS_IN, 5
Counter({5: 10000})


TRAIN EPOCH[001/150] LOSS:1.394996%

TRAIN EPOCH[001/150] ACC:20.230000%

TEST EPOCH[001/150] ACC:20.360000%

TRAIN EPOCH[002/150] LOSS:0.946825%

TRAIN EPOCH[002/150] ACC:49.570000%

TEST EPOCH[002/150] ACC:48.220000%

TRAIN EPOCH[003/150] LOSS:0.750114%

TRAIN EPOCH[003/150] ACC:54.560000%

TEST EPOCH[003/150] ACC:52.030000%

TRAIN EPOCH[004/150] LOSS:0.666048%

TRAIN EPOCH[004/150] ACC:61.120000%

TEST EPOCH[004/150] ACC:57.620000%

TRAIN EPOCH[005/150] LOSS:0.558474%

TRAIN EPOCH[005/150] ACC:61.820000%

TEST EPOCH[005/150] ACC:57.230000%

TRAIN EPOCH[006/150] LOSS:0.515673%

TRAIN EPOCH[006/150] ACC:62.390000%

TEST EPOCH[006/150] ACC:57.170000%

TRAIN EPOCH[007/150] LOSS:0.496837%

TRAIN EPOCH[007/150] ACC:65.870000%

TEST EPOCH[007/150] ACC:60.810000%

TRAIN EPOCH[008/150] LOSS:0.462659%

TRAIN EPOCH[008/150] ACC:63.680000%

TEST EPOCH[008/150] ACC:58.270000%

TRAIN EPOCH[009/150] LOSS:0.476499%

TRAIN EPOCH[009/150] ACC:63.630000%

TEST EPOCH[009/150] ACC:58.230000%

TRAIN EPOC

TRAIN EPOCH[076/150] LOSS:0.071763%

TRAIN EPOCH[076/150] ACC:78.220000%

TEST EPOCH[076/150] ACC:67.740000%

TRAIN EPOCH[077/150] LOSS:0.063658%

TRAIN EPOCH[077/150] ACC:77.800000%

TEST EPOCH[077/150] ACC:66.640000%

TRAIN EPOCH[078/150] LOSS:0.065216%

TRAIN EPOCH[078/150] ACC:77.780000%

TEST EPOCH[078/150] ACC:67.070000%

TRAIN EPOCH[079/150] LOSS:0.067289%

TRAIN EPOCH[079/150] ACC:78.470000%

TEST EPOCH[079/150] ACC:68.400000%

TRAIN EPOCH[080/150] LOSS:0.063966%

TRAIN EPOCH[080/150] ACC:78.440000%

TEST EPOCH[080/150] ACC:68.100000%

TRAIN EPOCH[081/150] LOSS:0.064923%

TRAIN EPOCH[081/150] ACC:78.750000%

TEST EPOCH[081/150] ACC:68.350000%

TRAIN EPOCH[082/150] LOSS:0.059315%

TRAIN EPOCH[082/150] ACC:77.910000%

TEST EPOCH[082/150] ACC:67.020000%

TRAIN EPOCH[083/150] LOSS:0.071549%

TRAIN EPOCH[083/150] ACC:78.700000%

TEST EPOCH[083/150] ACC:68.150000%

TRAIN EPOCH[084/150] LOSS:0.064521%

TRAIN EPOCH[084/150] ACC:77.920000%

TEST EPOCH[084/150] ACC:67.360000%

TRAIN EPOC

In [ ]:
0.36
0.41
0.45